In [1]:
import pandas as pd
import torch
from sentence_transformers import SentenceTransformer, util
import ir_datasets
import random

# Collecting data

In [2]:
dataset = ir_datasets.load("neuclir/1/multi/trec-2023")

In [ ]:
documents_df = pd.read_csv('sbert_documents.csv')
qrels_df = pd.DataFrame(dataset.qrels_iter())
queries_df = pd.DataFrame(dataset.queries_iter())
qrels_doc_ids = set(qrels_df["doc_id"])

In [4]:
documents_df.head()

,id,title,content,title_embedding,content_embedding
0,7707eaec-e3a6-422a-9e49-68ef7d2baaec,健康：身體和精神健康相互作用和影響背後的原理,健康：身體和精神健康相互作用和影響背後的原理\n\n10 分钟前\n\n圖像來源，Getty...,"tensor([ 1.7091e-01, 1.1061e-01, 7.6357e-03,...","tensor([ 0.2471, -0.0563, 0.1989, 0.2372, 0..."
1,27a8ca18-15fa-4f80-af10-d0fa53968995,尼羅河水爭奪戰越演越烈 聯合國安理會將召開會議,埃塞俄比亞復興大壩第二期蓄水工程遭到藍尼羅河下游國家蘇丹與埃及的強烈反對，聯合國安理會在突尼...,"tensor([ 1.7681e-01, 1.0508e-01, -6.0566e-02,...","tensor([-0.1660, 0.1614, -0.0299, -0.0677, 0..."
2,2e7d5130-d497-4f8c-9132-a4dc22ca6ce8,Visa：旗下虛擬貨幣卡上半年使用量逾10億美元,Visa Inc.財務總監Vasant Prabhu周三(7日)表示，今年上半年，旗下虛擬貨...,"tensor([ 1.7375e-01, 2.0704e-02, -2.7007e-01,...","tensor([ 1.2213e-01, -5.1399e-02, -1.6382e-01,..."
3,fdc63781-a14a-47c5-8518-8774df8d0e51,大蛇潛馬桶 偷襲如廁男,奧地利一名六十五歲男子在家中上廁所時，突遭到一條一點六米長的蟒蛇咬傷下體，所幸只受到輕傷，沒...,"tensor([-0.1478, 0.1569, 0.1280, -0.1398, -0...","tensor([-1.6577e-01, 1.3552e-01, 1.7160e-01,..."
4,f55f74b2-433b-478f-9b1b-158eb7cfa783,野火烟雾缭绕，BC 省专家支招如何在糟糕空气中保护自己,BC 省南部内陆大部分地区已经发出空气质量警告，而且 BC 省的火灾风险一直处于高位。\n\...,"tensor([ 0.0198, 0.4940, -0.1451, 0.1459, 0...","tensor([ 0.0918, 0.2722, 0.0957, 0.1512, 0..."


In [5]:
print(queries_df.shape[0])
queries_df.head()

76


,query_id,title,description,narrative,fa_mt_title,fa_mt_description,fa_mt_narrative,ru_mt_title,ru_mt_description,ru_mt_narrative,zh_mt_title,zh_mt_description,zh_mt_narrative
0,200,Corruption Bribery Sports Federation Olympics,Are there cases of Institutional Corruption an...,We are Looking for articles that contain a cas...,المپیک فدراسیون ورزشی رشوه خواری فساد,آیا مواردی از فساد نهادی و رشوه خواری در ورزش ...,ما به دنبال مقالاتی هستیم که حاوی یک مورد فساد...,Олимпийские игры Федерации спортивных игр корр...,Существуют ли случаи институциональной коррупц...,"Мы ищем статьи, которые содержат случай финанс...",腐败贿赂体育联合会奥运会,体育中是否存在机构腐败和贿赂的案例？,我们正在寻找在机构层面上包含金融腐败或贿赂案件的文章。奥运会和体育联合会中的贿赂案件是相关的...
1,201,China investment in Iran,In what fields or industries China’s investmen...,We need articles that specify at least one fie...,سرمایه گذاری چین در ایران,در کدام زمینه ها یا صنایع سرمایه گذاری چین در ...,ما به مقالاتی نیاز داریم که حداقل یک زمینه سرم...,Китайские инвестиции в Иран,В каких областях или промышленности инвестиции...,"Нам нужны статьи, в которых указано хотя бы од...",中国对伊朗的投资,在哪些领域或行业中，中国对伊朗的投资是？,我们需要至少指定一个投资领域的文章，无论是皮带和道路倡议，石油，水坝等。货币的投资金额不提及...
2,202,"Emerging technologies, precision farming, smar...",What are some of the latest emerging technolog...,Find articles on emerging technologies in prec...,فن آوری های نوظهور ، کشاورزی دقیق ، کشاورزی هو...,برخی از جدیدترین فن آوری های نوظهور که صنعت کش...,مقالاتی در مورد فن آوری های نوظهور در کشاورزی ...,"Новые технологии, точное сельское хозяйство, у...",Каковы некоторые из последних новых технологий...,Найдите статьи о новых технологиях в области т...,新兴技术，精确农业，智能农业，农业,哪些最新的新兴技术正在改变农业行业？,查找有关精确农业和农业中新兴技术的文章，这些文章提供了这些技术如何用于提高农作物产量并提高农...
3,203,Ever Given's Stuck,Find information about the ship Ever Given bei...,Find articles on the event when the container ...,تا به حال گیر کرده است,اطلاعات مربوط به کشتی را که در کانال سوئز گیر ...,مقالاتی را در مورد این رویداد پیدا کنید که کشت...,Когда -либо дал застрял,"Найдите информацию о корабле, когда -либо заст...","Найдите статьи на мероприятии, когда контейнер...",曾经被卡住了,查找有关被困在苏伊士运河的船的信息,在2021年春季遇到的集装箱船被困在苏伊士运河中时，请查找有关事件的事件的文章。包括有关事件...
4,204,Penalty doping sports stop,What strategies are used to reduce/stop doping...,Any mention of the monetary penalties or exclu...,مجازات دوپینگ ورزش توقف,از چه استراتژی هایی برای کاهش/متوقف کردن دوپین...,هرگونه ذکر مجازات های پولی یا محرومیت تیم ها ی...,Спортивная остановка пенальти,Какие стратегии используются для уменьшения/ос...,Любое упоминание о денежных наказаниях или иск...,点球兴奋剂运动停止,哪些策略用于减少运动中的掺杂？,包括任何提及货币罚款或排除在团队中或夺回奖牌的罚款。接受掺杂的测试将不被视为预防措施。


In [6]:
qrels_df.head()

,query_id,doc_id,relevance,iteration
0,200,00258365-6d48-49cc-901f-578b883c8226,1,fas
1,200,010fd82c-4423-41a6-ac56-4d036ccf0524,2,fas
2,200,013c3243-d1b5-47e6-bcf5-26c092ac9ff5,0,fas
3,200,01aed8a4-f1b9-4729-97f0-525338029268,0,fas
4,200,01e59322-b610-4d44-9ecd-81bfad2f2b5e,0,fas


# Ranking Documents

### Load model

In [ ]:
model_name = "sentence-transformers/paraphrase-MiniLM-L6-v2"
st_model = SentenceTransformer(model_name)

### Genearte embeddings

In [8]:
from torch.utils.data import DataLoader

def generate_embeddings(texts, model, batch_size=32):
    embeddings = []
    # for handling batching
    dataloader = DataLoader(texts, batch_size=batch_size, shuffle=False)
    with torch.no_grad():
        for batch_texts in dataloader:
            # Encode the batch and collect embeddings
            batch_embeddings = model.encode(batch_texts, convert_to_tensor=True, show_progress_bar=False)
            embeddings.append(batch_embeddings)
    
    return torch.cat(embeddings, dim=0)

In [ ]:
# Generate embeddings for queries
query_texts = queries_df['title'].tolist()
query_embeddings = generate_embeddings(query_texts, st_model)

# Generate embeddings for documents
doc_contents = documents_df['content'].tolist()
doc_ids = documents_df['id'].tolist()
doc_embeddings = generate_embeddings(doc_contents, st_model)

Generating query embeddings...


### Rank Documents

In [ ]:
from tqdm import tqdm
from sentence_transformers import util

query_embeddings = query_embeddings.cpu()
doc_content_embeddings = doc_content_embeddings.cpu()

ranked_results = {}

for idx in tqdm(range(len(query_embeddings)), total=len(query_embeddings), desc="Processing queries"):
    query_id = queries_df.iloc[idx]['query_id']
    
    # Get the query embedding from the tensor
    query_embedding = query_embeddings[idx]
    
    # Ensure the query_embedding has the correct shape
    if len(query_embedding.shape) == 1:
        query_embedding = query_embedding.unsqueeze(0)
    
    # Calculate cosine similarities
    similarities = util.cos_sim(query_embedding, doc_content_embeddings)
    similarities = similarities[0]
    
    ranked_indices = similarities.argsort(descending=True)
    ranked_results[query_id] = [{"doc_id": doc_ids[i], "score": similarities[i].item()} for i in ranked_indices]

print(f"Processed ranking for {len(ranked_results)} queries.")


Processing queries: 100%|██████████| 76/76 [00:17<00:00,  4.36it/s]

Processed ranking for 76 queries.


In [17]:
# Print the first 5
for query_id, results in list(ranked_results.items())[:5]:
    print(f"Query ID: {query_id}")
    for result in results[:5]:  # Print top 5 results for each query
        print(f"  Doc ID: {result['doc_id']}, Score: {result['score']}")

Query ID: 200
  Doc ID: b7d9703b-bbe4-46e4-8b2f-1e4c4ef4ea20, Score: 0.555435061454773
  Doc ID: c8b66c15-d538-4f8a-9534-f5adfca6188c, Score: 0.527833104133606
  Doc ID: 99a91ff3-c6f9-43bc-8963-2e21468d6b20, Score: 0.5186973214149475
  Doc ID: fb49ace6-aaa3-4ce0-a6bb-5504213cac27, Score: 0.5180222392082214
  Doc ID: c6fa30d0-dbfc-4bc3-892c-5e95a65f46b1, Score: 0.515623927116394
Query ID: 201
  Doc ID: d865da36-55e8-4500-a676-32b7a7ffbe11, Score: 0.6288976073265076
  Doc ID: 97524236-14b3-4a5c-a0dc-5c9326745140, Score: 0.6276373863220215
  Doc ID: 0b656f7f-40bc-4df0-8b5c-82cbb5c22834, Score: 0.6238518953323364
  Doc ID: d7dfd276-e1dd-412d-8d2e-dda97412a80a, Score: 0.621002733707428
  Doc ID: 8bb7673a-3292-4e2d-9541-d8959d87d480, Score: 0.6117416620254517
Query ID: 202
  Doc ID: d0b160f5-f4e6-4fa9-88e9-0e88ec4ccd1f, Score: 0.5949203968048096
  Doc ID: e86e2307-a04b-4268-a600-b780a4223b8d, Score: 0.5854308605194092
  Doc ID: 3e07b790-0e75-4a0a-bceb-5bfcce84a7f3, Score: 0.5725286602973938


## Evaluation

In [19]:
valid_doc_ids = set(documents_df['id'])
qrels_df = pd.DataFrame(dataset.qrels_iter())

In [20]:
import ir_measures
from ir_measures import ScoredDoc, Qrel


for query_id, docs in ranked_results.items():
    ranked_results[query_id] = sorted(docs, key=lambda x: x['score'], reverse=True)

qrels = [
    Qrel(query_id=row['query_id'], doc_id=row['doc_id'], relevance=int(row['relevance']))
    for _, row in qrels_df.iterrows()
]

results = [
    ScoredDoc(query_id=query_id, doc_id=doc['doc_id'], score=float(doc['score']))
    for query_id, docs in ranked_results.items()
    for doc in docs
]

metrics = [
    ir_measures.nDCG @ 20,   # nDCG@20
    ir_measures.AP,          # Average Precision
    ir_measures.RBP(rel=1),  # RBP with relevance=1
    ir_measures.R @ 100,     # Recall@100
    ir_measures.R @ 1000     # Recall@1000
]

evaluation = ir_measures.calc_aggregate(metrics, qrels, results)

print("Evaluation Metrics:")
for metric, value in evaluation.items():
    print(f"{metric}: {value:.4f}")


Evaluation Metrics:
AP: 0.2116
RBP(rel=1): 0.4230
nDCG@20: 0.2915
R@100: 0.2158
R@1000: 0.6472
